In [1]:
import pyspark
sc = pyspark.SparkContext(appName="myAppName")
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession(sc)

In [2]:
# DataFrame a partir de un RDD de listas/tuplas infiriendo el esquema
# Leemos el fichero apat63_99.txt.
rdd = sc.textFile("./datos/apat63_99.txt").cache()
# Le quitamos la cabecera y lo convertimos en un RDD de listas
rddSplit = rdd.filter(lambda l: not l.startswith('"PATENT"'))\
              .map(lambda l: l.split(",")[0:16])
print(rddSplit.take(10))
# Obtengo la cabecera como una lista de nombres (sin comillas dobles)
cabecera = [c.strip('"') for c in rdd.take(1)[0].split(",")[0:16]]
# print('------SE TOMAN LAS 2 PRIMERAS LINEAS DEL DOCUMENTO------')
# print(rdd.take(2))
# ['"PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"', '3070801,1963,1096,,"BE","",,1,,269,6,69,,1,,0,,,,,,,']
# print('------SE TOMA EL OBJETO 0 DEL ARRAY------')
# print(rdd.take(1)[0])
# "PATENT","GYEAR","GDATE","APPYEAR","COUNTRY","POSTATE","ASSIGNEE","ASSCODE","CLAIMS","NCLASS","CAT","SUBCAT","CMADE","CRECEIVE","RATIOCIT","GENERAL","ORIGINAL","FWDAPLAG","BCKGTLAG","SELFCTUB","SELFCTLB","SECDUPBD","SECDLWBD"
# print('------SE TOMA LOS PRIMEROS 16 CAMPOS DEL OBJETO 0 DEL ARRAY------')
# print(rdd.take(1)[0].split(",")[0:16])
# ['"PATENT"', '"GYEAR"', '"GDATE"', '"APPYEAR"', '"COUNTRY"', '"POSTATE"', '"ASSIGNEE"', '"ASSCODE"', '"CLAIMS"', '"NCLASS"', '"CAT"', '"SUBCAT"', '"CMADE"', '"CRECEIVE"', '"RATIOCIT"', '"GENERAL"']
print('------SE IMPRIME LA CABECERA------')
print(cabecera)
rdd.unpersist()
rddSplit.cache()

[['3070801', '1963', '1096', '', '"BE"', '""', '', '1', '', '269', '6', '69', '', '1', '', '0'], ['3070802', '1963', '1096', '', '"US"', '"TX"', '', '1', '', '2', '6', '63', '', '0', '', ''], ['3070803', '1963', '1096', '', '"US"', '"IL"', '', '1', '', '2', '6', '63', '', '9', '', '0.3704'], ['3070804', '1963', '1096', '', '"US"', '"OH"', '', '1', '', '2', '6', '63', '', '3', '', '0.6667'], ['3070805', '1963', '1096', '', '"US"', '"CA"', '', '1', '', '2', '6', '63', '', '1', '', '0'], ['3070806', '1963', '1096', '', '"US"', '"PA"', '', '1', '', '2', '6', '63', '', '0', '', ''], ['3070807', '1963', '1096', '', '"US"', '"OH"', '', '1', '', '623', '3', '39', '', '3', '', '0.4444'], ['3070808', '1963', '1096', '', '"US"', '"IA"', '', '1', '', '623', '3', '39', '', '4', '', '0.375'], ['3070809', '1963', '1096', '', '"US"', '"AZ"', '', '1', '', '4', '6', '65', '', '0', '', ''], ['3070810', '1963', '1096', '', '"US"', '"IL"', '', '1', '', '4', '6', '65', '', '3', '', '0.4444']]
------SE IMPRI

PythonRDD[4] at RDD at PythonRDD.scala:53

In [3]:
# Dos formas de crear el DataFrame
# 1. A partir del método createDataFrame de sqlContext
sqlContext = SQLContext(sc)
dfInfer1 = sqlContext.createDataFrame(rddSplit, cabecera)
# 2. A partir del método toDF del RDD
dfInfer2 = rddSplit.toDF(cabecera)
dfInfer1.show(10)
dfInfer2.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
|3070801| 1963| 1096|       |   "BE"|     ""|        |      1|      |   269|  6|    69|     |       1|        |      0|
|3070802| 1963| 1096|       |   "US"|   "TX"|        |      1|      |     2|  6|    63|     |       0|        |       |
|3070803| 1963| 1096|       |   "US"|   "IL"|        |      1|      |     2|  6|    63|     |       9|        | 0.3704|
|3070804| 1963| 1096|       |   "US"|   "OH"|        |      1|      |     2|  6|    63|     |       3|        | 0.6667|
|3070805| 1963| 1096|       |   "US"|   "CA"|        |      1|      |     2|  6|    63|     |       1|        |      0|
|3070806| 1963| 1096|       |   "US"|   

In [4]:
# Esquema de la tabla
dfInfer2.printSchema()

root
 |-- PATENT: string (nullable = true)
 |-- GYEAR: string (nullable = true)
 |-- GDATE: string (nullable = true)
 |-- APPYEAR: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- POSTATE: string (nullable = true)
 |-- ASSIGNEE: string (nullable = true)
 |-- ASSCODE: string (nullable = true)
 |-- CLAIMS: string (nullable = true)
 |-- NCLASS: string (nullable = true)
 |-- CAT: string (nullable = true)
 |-- SUBCAT: string (nullable = true)
 |-- CMADE: string (nullable = true)
 |-- CRECEIVE: string (nullable = true)
 |-- RATIOCIT: string (nullable = true)
 |-- GENERAL: string (nullable = true)



In [5]:
# Los tipos de datos no se han inferido de forma correcta
# Para que los tipos se infieran correctamente, podemos partir de un RDD de listas con los tipos correctos para cada campo.
# Convierto cambio el tipo de los datos del RDD de listas
def toIntSafe(inval):
  try:
    return int(inval)
  except ValueError:
    return 0
    
def toFloatSafe(inval):
  try:
    return float(inval)
  except ValueError:
    return 0.0

# Dejo todos los campos como Strings, menos el campo 8 (CLAIMS) que lo pongo como entero
# y el campo 15 (GENERAL) que lo pongo como float
rddTipos = rddSplit.map(lambda l: (l[0], 
                                   l[1],
                                   l[2], 
                                   l[3], 
                                   l[4].strip('"'), 
                                   l[5].strip('"'), 
                                   l[6], 
                                   l[7], 
                                   toIntSafe(l[8]),
                                   l[9],
                                   l[10], 
                                   l[11], 
                                   l[12], 
                                   l[13], 
                                   l[14], 
                                   toFloatSafe(l[15])))
rddTipos.cache()
dfInfer3 = sqlContext.createDataFrame(rddTipos, cabecera)
dfInfer3.printSchema()
dfInfer3.show(10)

root
 |-- PATENT: string (nullable = true)
 |-- GYEAR: string (nullable = true)
 |-- GDATE: string (nullable = true)
 |-- APPYEAR: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- POSTATE: string (nullable = true)
 |-- ASSIGNEE: string (nullable = true)
 |-- ASSCODE: string (nullable = true)
 |-- CLAIMS: long (nullable = true)
 |-- NCLASS: string (nullable = true)
 |-- CAT: string (nullable = true)
 |-- SUBCAT: string (nullable = true)
 |-- CMADE: string (nullable = true)
 |-- CRECEIVE: string (nullable = true)
 |-- RATIOCIT: string (nullable = true)
 |-- GENERAL: double (nullable = true)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
|3070801| 1963| 1096|  

In [6]:
# DataFrame a partir de un RDD de listas/tuplas indicando el esquema de forma explícita
# Defino el esquema para los elementos de la tabla usando un StructType de StructField
# StructType: Permite definir un esquema para el DataFrame a partir de una lista de StructFields
# StructField: Definen el nombre y tipo de cada columna, así como si es nullable o no
from pyspark.sql.types import *
# Defino el esquema para los elementos de la tabla
# StructType -> Permite definir un esquema para el DF a partir de una lista de StructFields
# StructField -> Definen el nombre y tipo de cada columna, así como si es nullable o no (campo True)
postSchema = StructType([
  StructField(cabecera[0], StringType(), False),
  StructField(cabecera[1], StringType(), True),
  StructField(cabecera[2], StringType(), True),
  StructField(cabecera[3], StringType(), True),
  StructField(cabecera[4], StringType(), True),
  StructField(cabecera[5], StringType(), True),
  StructField(cabecera[6], StringType(), True),
  StructField(cabecera[7], StringType(), True),
  StructField(cabecera[8], IntegerType(), True),
  StructField(cabecera[9], StringType(), True),
  StructField(cabecera[10], StringType(), True),
  StructField(cabecera[11], StringType(), True),
  StructField(cabecera[12], StringType(), False),
  StructField(cabecera[13], StringType(), True),
  StructField(cabecera[14], StringType(), True),
  StructField(cabecera[15], FloatType(), True)
  ])
# Creo el DataFrame
dfSchema = sqlContext.createDataFrame(rddTipos, postSchema).cache()
rddTipos.unpersist()
dfSchema.printSchema()
dfSchema.show(10)

root
 |-- PATENT: string (nullable = false)
 |-- GYEAR: string (nullable = true)
 |-- GDATE: string (nullable = true)
 |-- APPYEAR: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- POSTATE: string (nullable = true)
 |-- ASSIGNEE: string (nullable = true)
 |-- ASSCODE: string (nullable = true)
 |-- CLAIMS: integer (nullable = true)
 |-- NCLASS: string (nullable = true)
 |-- CAT: string (nullable = true)
 |-- SUBCAT: string (nullable = true)
 |-- CMADE: string (nullable = false)
 |-- CRECEIVE: string (nullable = true)
 |-- RATIOCIT: string (nullable = true)
 |-- GENERAL: float (nullable = true)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+
|3070801| 1963| 109

In [7]:
# DataFrame a partir de un RDD de objetos Row
# Row Representa una fila de datos en un DataFrame
from pyspark.sql import Row
# Convierto el RDD de listas en un RDD de objetos Row
rddRows = rddSplit.map(lambda l: Row(Patent = l[0], 
                                     Gyear = l[1], 
                                     Gdate = l[2], 
                                     Appyear = l[3],
                                     Country = l[4],
                                     Postate = l[5],
                                     Assignee = l[6], 
                                     Asscode = l[7],
                                     Claims = toIntSafe(l[8]),
                                     Nclass = l[9], 
                                     Cat = l[10], 
                                     Subcat = l[11], 
                                     Cmade = l[12],
                                     Creceive = l[13],
                                     Ratiocit = l[14],
                                     General = toFloatSafe(l[15])))
# El esquema se infiere de los tipos
dfRows = sqlContext.createDataFrame(rddRows)
print("Esquema de la tabla en árbol")
dfRows.printSchema()
print("Nombres de las columnas\n{0}\n".
      format(dfRows.columns))
print("Tipos de las columnas\n{0}\n".
      format(dfRows.dtypes))
rddSplit.unpersist()
dfRows.show(10)

Esquema de la tabla en árbol
root
 |-- Appyear: string (nullable = true)
 |-- Asscode: string (nullable = true)
 |-- Assignee: string (nullable = true)
 |-- Cat: string (nullable = true)
 |-- Claims: long (nullable = true)
 |-- Cmade: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Creceive: string (nullable = true)
 |-- Gdate: string (nullable = true)
 |-- General: double (nullable = true)
 |-- Gyear: string (nullable = true)
 |-- Nclass: string (nullable = true)
 |-- Patent: string (nullable = true)
 |-- Postate: string (nullable = true)
 |-- Ratiocit: string (nullable = true)
 |-- Subcat: string (nullable = true)

Nombres de las columnas
['Appyear', 'Asscode', 'Assignee', 'Cat', 'Claims', 'Cmade', 'Country', 'Creceive', 'Gdate', 'General', 'Gyear', 'Nclass', 'Patent', 'Postate', 'Ratiocit', 'Subcat']

Tipos de las columnas
[('Appyear', 'string'), ('Asscode', 'string'), ('Assignee', 'string'), ('Cat', 'string'), ('Claims', 'bigint'), ('Cmade', 'string'), ('Countr

In [8]:
# Conversion de un DataFrame en un RDD de objetos Row
# Permite convertir un DataFrame en un RDD
rddRows2 = dfSchema.rdd
print("Muestra un elemento del nuevo RDD")
print(rddRows2.take(2))
print("Aplicamos un map al RDD")
print(rddRows2.map(lambda r: (r.COUNTRY, r.PATENT, r.GYEAR)).take(2))

Muestra un elemento del nuevo RDD
[Row(PATENT='3070801', GYEAR='1963', GDATE='1096', APPYEAR='', COUNTRY='BE', POSTATE='', ASSIGNEE='', ASSCODE='1', CLAIMS=0, NCLASS='269', CAT='6', SUBCAT='69', CMADE='', CRECEIVE='1', RATIOCIT='', GENERAL=0.0), Row(PATENT='3070802', GYEAR='1963', GDATE='1096', APPYEAR='', COUNTRY='US', POSTATE='TX', ASSIGNEE='', ASSCODE='1', CLAIMS=0, NCLASS='2', CAT='6', SUBCAT='63', CMADE='', CRECEIVE='0', RATIOCIT='', GENERAL=0.0)]
Aplicamos un map al RDD
[('BE', '3070801', '1963'), ('US', '3070802', '1963')]


In [9]:
# DataFrame a partir de un fichero JSON
dfJson = sqlContext.read.json("./datos/gente.json")
dfJson.show()

import os
os.system("rmdir /S /Q salida\\apat63_99-json")
# Guardar el DataFrame como fichero JSON
dfSchema.write.json("./salida/apat63_99-json")

+----+-------+
|edad| nombre|
+----+-------+
|  17|  Celia|
|  53|   Juan|
|  39|Manuela|
|  17|    Ana|
+----+-------+



In [10]:
##### Operaciones básicas
##### Selección y eliminación de columnas
dfParcial = dfSchema.select("PATENT", "GYEAR", "COUNTRY", "CLAIMS")
dfParcial.show(10)
print("El objeto dfParcial es de tipo {0}".format(type(dfParcial)))
dfSchema.unpersist()
dfParcial.cache()
# También es posible crear objetos de tipo Column
colPatent = dfParcial["PATENT"]
colCountry = dfParcial.COUNTRY
print("El objeto colPatent es de tipo {0}".format(type(colPatent)))
print("El objeto colCountry es de tipo {0}".format(type(colCountry)))
# Y crear un DataFrame a partir de objetos Column, renombrando columnas
dfParcial2 = dfParcial.select(colPatent.alias("Patente"), colCountry.alias("País"), dfParcial.GYEAR.alias("Año"))
print(dfParcial.printSchema())
dfParcial2.show()
# Se pueden eliminar columnas
dfParcial3 = dfParcial.drop("CLAIMS")
dfParcial3.show(10)

+-------+-----+-------+------+
| PATENT|GYEAR|COUNTRY|CLAIMS|
+-------+-----+-------+------+
|3070801| 1963|     BE|     0|
|3070802| 1963|     US|     0|
|3070803| 1963|     US|     0|
|3070804| 1963|     US|     0|
|3070805| 1963|     US|     0|
|3070806| 1963|     US|     0|
|3070807| 1963|     US|     0|
|3070808| 1963|     US|     0|
|3070809| 1963|     US|     0|
|3070810| 1963|     US|     0|
+-------+-----+-------+------+
only showing top 10 rows

El objeto dfParcial es de tipo <class 'pyspark.sql.dataframe.DataFrame'>
El objeto colPatent es de tipo <class 'pyspark.sql.column.Column'>
El objeto colCountry es de tipo <class 'pyspark.sql.column.Column'>
root
 |-- PATENT: string (nullable = false)
 |-- GYEAR: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- CLAIMS: integer (nullable = true)

None
+-------+----+----+
|Patente|País| Año|
+-------+----+----+
|3070801|  BE|1963|
|3070802|  US|1963|
|3070803|  US|1963|
|3070804|  US|1963|
|3070805|  US|1963|
|307080

In [11]:
##### Filtrado
# Patentes con CLAIMS > 0
dfClaims = dfParcial.where('CLAIMS > 0')
print("Número de patentes con reivindicaciones: {0}\n".\
       format(dfClaims.count()))
dfClaims.show(1)
# Patentes con inventor español
dfEsp = dfParcial.filter(colCountry.like('ES'))
print("Número de patentes con inventor español: {0}\n".\
       format(dfEsp.count()))
dfEsp.show(1)

Número de patentes con reivindicaciones: 1984055

+-------+-----+-------+------+
| PATENT|GYEAR|COUNTRY|CLAIMS|
+-------+-----+-------+------+
|3858241| 1975|     US|     6|
+-------+-----+-------+------+
only showing top 1 row

Número de patentes con inventor español: 3601

+-------+-----+-------+------+
| PATENT|GYEAR|COUNTRY|CLAIMS|
+-------+-----+-------+------+
|3071250| 1963|     ES|     0|
+-------+-----+-------+------+
only showing top 1 row



In [12]:
##### Ordenación y agrupamiento
dfParcial.orderBy('CLAIMS', ascending=False).show(10)
grupoPorPais = dfParcial.groupBy('COUNTRY')
print(type(grupoPorPais))
print("Número de patentes por país")
grupoPorPais.count().orderBy('count', ascending=False).show()
print("Media de reivindicaciones por país")
grupoPorPais.avg('CLAIMS').orderBy('COUNTRY').show()

+-------+-----+-------+------+
| PATENT|GYEAR|COUNTRY|CLAIMS|
+-------+-----+-------+------+
|5095054| 1992|     DE|   868|
|4085139| 1978|     US|   706|
|4373527| 1983|     US|   642|
|4081478| 1978|     US|   472|
|4624109| 1986|     US|   394|
|4068298| 1978|     US|   393|
|5812453| 1998|     JP|   348|
|4533693| 1985|     US|   346|
|5802314| 1998|     US|   320|
|4905163| 1990|     US|   313|
+-------+-----+-------+------+
only showing top 10 rows

<class 'pyspark.sql.group.GroupedData'>
Número de patentes por país
+-------+-------+
|COUNTRY|  count|
+-------+-------+
|     US|1784989|
|     JP| 421441|
|     DE| 221095|
|     GB|  98012|
|     FR|  85398|
|     CA|  53872|
|     CH|  43313|
|     IT|  32433|
|     SE|  28286|
|     NL|  26687|
|     TW|  19979|
|     KR|  14855|
|     AU|  11386|
|     BE|  10972|
|     AT|  10260|
|     IL|   7378|
|     SU|   6992|
|     FI|   6984|
|     DK|   6479|
|     ES|   3601|
+-------+-------+
only showing top 20 rows

Media de reivi

In [13]:
# Obtener el máximo, mínimo, media y desviación estándard de las reivindicaciones de las patentes españolas
from pyspark.sql.functions import *
dfEsp.select(max("CLAIMS"), min("CLAIMS"),avg("CLAIMS"),stddev("CLAIMS")).show()

# Otra forma de hacer lo mismo
dfEsp.describe("CLAIMS").show()

##### Consultas SQL
# Registra la tabla para usar SQL
dfParcial.registerTempTable("patentinfo")
sqlContext.sql("SELECT COUNTRY,CLAIMS FROM patentinfo WHERE CLAIMS >= 100").show()

+-----------+-----------+-----------------+-------------------+
|max(CLAIMS)|min(CLAIMS)|      avg(CLAIMS)|stddev_samp(CLAIMS)|
+-----------+-----------+-----------------+-------------------+
|         96|          0|6.582615940016662|  7.307029943787108|
+-----------+-----------+-----------------+-------------------+

+-------+-----------------+
|summary|           CLAIMS|
+-------+-----------------+
|  count|             3601|
|   mean|6.582615940016662|
| stddev|7.307029943787108|
|    min|                0|
|    max|               96|
+-------+-----------------+

+-------+------+
|COUNTRY|CLAIMS|
+-------+------+
|     US|   104|
|     FR|   127|
|     US|   101|
|     US|   103|
|     DE|   111|
|     DE|   108|
|     DE|   115|
|     US|   147|
|     DE|   176|
|     US|   119|
|     US|   139|
|     US|   112|
|     US|   109|
|     US|   127|
|     US|   135|
|     US|   132|
|     US|   103|
|     US|   212|
|     US|   160|
|     US|   106|
+-------+------+
only showing top 2

In [14]:
##### UDFs: Funciones definidas por el usuario
from pyspark.sql.functions import udf
from pyspark.sql.types import BooleanType
esPar = udf(lambda n: not n%2, BooleanType())
print("Información sobre si el número de reivindicaciones es par o impar.")
dfParcial.select(dfParcial.PATENT, dfParcial.CLAIMS, esPar(dfParcial.CLAIMS).alias("Par?")).\
  orderBy(dfParcial.CLAIMS, ascending=False).show()

Información sobre si el número de reivindicaciones es par o impar.
+-------+------+-----+
| PATENT|CLAIMS| Par?|
+-------+------+-----+
|5095054|   868| true|
|4085139|   706| true|
|4373527|   642| true|
|4081478|   472| true|
|4624109|   394| true|
|4068298|   393|false|
|5812453|   348| true|
|4533693|   346| true|
|5802314|   320| true|
|4905163|   313|false|
|5673402|   309|false|
|4847639|   307|false|
|5190690|   306| true|
|4544239|   298| true|
|5825664|   293|false|
|4996627|   292| true|
|5786333|   288| true|
|5007353|   279|false|
|4666252|   279|false|
|5625670|   276| true|
+-------+------+-----+
only showing top 20 rows



In [15]:
sc.stop()